In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from db.models import get_restaurants_with_reviews_and_users
from sqlalchemy import Table, MetaData
from sqlalchemy.dialects.sqlite import insert


class NLPPretraitement:
    def __init__(self, db_url='sqlite:///restaurant_reviews.db'):
        # Connexion à la base de données
        self.engine = create_engine(db_url)
        self.Session = sessionmaker(bind=self.engine)
        self.session = self.Session()
        self.lemmatizer = WordNetLemmatizer()
        
        # Initialiser le DataFrame
        self.avis_restaurants = None

    def extraction_donnees(self):
        avis_restaurants_data = get_restaurants_with_reviews_and_users(self.session)
        data = []
        for restaurant in avis_restaurants_data:
            for review in restaurant['reviews']:
                data.append({
                    'restaurant': restaurant['restaurant'],
                    'restaurant_address': restaurant['restaurant_address'],
                    'title': review['title'],
                    'user_profile': review['user_profile'],
                    'date_review': review['date_review'],
                    'rating': review['rating'],
                    'type_visit': review['type_visit'],
                    'num_contributions': review['num_contributions'],
                    'review': review['review']
                })
        self.avis_restaurants = pd.DataFrame(data)

    def nettoyer_avis(self, avis):
        avis = avis.lower()
        avis = re.sub(r'[^\w\s]', '', avis)
        avis = re.sub(r'[0-9]', '', avis)
        tokens = word_tokenize(avis)
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in stopwords.words('french')]
        tokens = [word for word in tokens if len(word) > 2]
        return ' '.join(tokens)

    def appliquer_nettoyage(self):
        self.avis_restaurants['review_cleaned'] = self.avis_restaurants['review'].apply(self.nettoyer_avis)

    def vectorisation(self, methode='bow', max_features=500):
        vectorizer = CountVectorizer(max_features=max_features) if methode == 'bow' else TfidfVectorizer(max_features=max_features)
        vecteurs = vectorizer.fit_transform(self.avis_restaurants['review_cleaned'])
        return pd.DataFrame(vecteurs.toarray(), columns=vectorizer.get_feature_names_out())

    def afficher_dataframe_complet(self):
        return self.avis_restaurants



In [ ]:
# Exemple d'utilisation
pretraitement = NLPPretraitement()
pretraitement.extraction_donnees()
pretraitement.appliquer_nettoyage()

In [3]:
# Affichage final du DataFrame
df_final = pretraitement.afficher_dataframe_complet()
df_final

              restaurant                                 restaurant_address  \
0  Le Bouchon des Filles  20 Rue Sergent-Blandan Parking Sathonay, 69001...   
1  Le Bouchon des Filles  20 Rue Sergent-Blandan Parking Sathonay, 69001...   
2  Le Bouchon des Filles  20 Rue Sergent-Blandan Parking Sathonay, 69001...   
3  Le Bouchon des Filles  20 Rue Sergent-Blandan Parking Sathonay, 69001...   
4  Le Bouchon des Filles  20 Rue Sergent-Blandan Parking Sathonay, 69001...   

                                               title     user_profile  \
0                                  Très belle soirée           SetC77   
1  Vive la bonne cuisine dans une ambiance conviv...   H3293ZGsylviel   
2                                          Sans plus  marieno_lleb739   
3                                      Bon et joyeux          Vymsbmm   
4  Bon restaurant et endroit pour se retrouver en...       Youliic974   

  date_review  rating type_visit  num_contributions  \
0  2024-12-16     5.0    friend

,restaurant,restaurant_address,title,user_profile,date_review,rating,type_visit,num_contributions,review,review_cleaned
0,Le Bouchon des Filles,"20 Rue Sergent-Blandan Parking Sathonay, 69001...",Très belle soirée,SetC77,2024-12-16,5.0,friends,67,"Trop bon moment!! \nAccueil, plats, ambiance t...",trop bon moment accueil plat ambiance tout pas...
1,Le Bouchon des Filles,"20 Rue Sergent-Blandan Parking Sathonay, 69001...",Vive la bonne cuisine dans une ambiance conviv...,H3293ZGsylviel,2024-11-26,5.0,friends,2,"Toujours aussi goûteux !\nL'ambiance, la bonne...",toujours aussi goûteux lambiance bonne humeur ...
2,Le Bouchon des Filles,"20 Rue Sergent-Blandan Parking Sathonay, 69001...",Sans plus,marieno_lleb739,2024-11-23,3.0,friends,96,Les serveurs et serveuses sont sympas. Mais l’...,serveurs serveuses sympas longlet nest tendre ...
3,Le Bouchon des Filles,"20 Rue Sergent-Blandan Parking Sathonay, 69001...",Bon et joyeux,Vymsbmm,2024-11-14,5.0,friends,225,Une excellente soirée dans ce petit restaurant...,excellente soirée petit restaurant peu lécart ...
4,Le Bouchon des Filles,"20 Rue Sergent-Blandan Parking Sathonay, 69001...",Bon restaurant et endroit pour se retrouver en...,Youliic974,2024-11-01,4.0,friends,38,Un super moment entre amis.\nLes plats étaient...,super moment entre amis plat très délicieux co...
...,...,...,...,...,...,...,...,...,...,...
17155,Le Bouchon Sully,"20, rue Sully, 69006 Lyon France",C est variable,Del6901,2015-07-10,3.0,business,16,Ns sommes allés plusieures fois ds ce restaura...,somme allés plusieures fois restaurant bilan a...
17156,Le Bouchon Sully,"20, rue Sully, 69006 Lyon France",Très moyen,quantis38,2015-06-10,2.0,friends,11,Service très lent et pas réactif du tout. Cuis...,service très lent réactif tout cuisine correct...
17157,Le Bouchon Sully,"20, rue Sully, 69006 Lyon France",excellent,dlc2009lcd,2015-05-05,5.0,family,1,excellent bouchon lyonnais cuisine de qualite ...,excellent bouchon lyonnais cuisine qualite tre...
17158,Le Bouchon Sully,"20, rue Sully, 69006 Lyon France",excellent,303gaellep,2015-04-24,5.0,family,2,"Très bonne cuisine, service impeccable. Le ser...",très bonne cuisine service impeccable serveur ...
